In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas_read_xml as pdx
from pandas_read_xml import auto_separate_tables

In [3]:
df_2 = pdx.read_xml('export.xml', transpose=True)

In [4]:
df_2

,HealthData
@locale,en_US
ExportDate,{'@value': '2020-08-18 21:01:15 -0500'}
Me,{'@HKCharacteristicTypeIdentifierDateOfBirth':...
Record,"[{'@type': 'HKQuantityTypeIdentifierHeight', '..."


In [14]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, @locale to Record
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   HealthData  4 non-null      object
dtypes: object(1)
memory usage: 64.0+ bytes


In [5]:
def list_separate_tables(df: pd.DataFrame) -> list:
    potential_separate_table = list(set([
        '|'.join(column.split('|')[:-1])
        for column in df.columns
        if determine_flatten_action_for_column(df, column).__name__ != "do_nothing"
    ]))
    potential_separate_table.sort()

    separate_tables_list = potential_separate_table
    for i, potential in enumerate(potential_separate_table):
        if (i < len(potential_separate_table) - 1) and (potential_separate_table[i+1].startswith(potential)):
            separate_tables_list.remove(potential_separate_table[i+1])
    if separate_tables_list == ['']:
        return ['main']+potential_separate_table
    else:
        return [column for column in df.columns
                if determine_flatten_action_for_column(df, column).__name__ != "do_nothing"]

In [7]:
list_separate_tables(df_2)

NameError: name 'determine_flatten_action_for_column' is not defined

In [ ]:
data = df_2.pipe(auto_separate_tables, key_columns)

In [ ]:
#https://pypi.org/project/pandas-read-xml/

In [ ]:




columns = ["name", "email", "age"]

datatframe = pd.DataFrame(columns = columns)

for node in root: 

    name = node.attrib.get("name")

    mail = node.find("email").text if node is not None else None

    age = node.find("age").text if node is not None else None

    datatframe = datatframe.append(pd.Series([name, mail, age], index = columns), ignore_index = True)

In [ ]:
import xml.etree.ElementTree as etree

In [ ]:
tree = etree.parse("export.xml")

In [ ]:
root = tree.getroot()

In [ ]:
root.tag, root.attrib

In [ ]:
root[2]

In [ ]:
columns = ['creationDate', 'startDate', 'endDate', 'value']

In [ ]:
df = pd.DataFrame(columns = columns)

In [ ]:
for node in root: 

    creationDate = node.attrib.get("creationDate")

    startDate = node.find("startDate").text if node is not None else None

    endDate = node.find("endDate").text if node is not None else None
    
    value = node.find("value").text if node is not None else None

    df = datatframe.append(pd.Series([creationDate, startDate, endDate, value], index = columns), ignore_index = True)